## Como funciona o modelo Naive Bayes?
O Naive Bayes é um algoritmo de classificação baseado no Teorema de Bayes, que calcula a probabilidade de um evento ocorrer com base em informações anteriores. No caso do filtro de spam, queremos calcular:
<br>
<br>
$$
P(Spam∣Palavras da Mensagem)
$$
<br>
Ou seja, a probabilidade de uma mensagem ser spam, dado que ela contém certas palavras.

Para isso, precisamos calcular duas probabilidades:

P(Palavra | Spam) -> A chance de uma palavra aparecer em mensagens de spam.

P(Palavra | Não Spam) -> A chance da mesma palavra aparecer em mensagens normais.

## O conceito de "Ingênuo" no Naive Bayes
O nome Naive (ingênuo) vem de uma suposição forte: as palavras dentro de uma mensagem são consideradas independentes entre si.

Exemplo:

Sabemos que palavras como "bitcoin" e "rolex" aparecem frequentemente em spams.
No mundo real, se uma mensagem contém "bitcoin", é provável que "rolex" também apareça.
Mas no Naive Bayes, essas palavras são tratadas como independentes.
Matematicamente, isso significa que:
<br>
<br>
$$
P("bitcoin" e "rolex"∣Spam) = P("bitcoin"∣Spam) × P("rolex"∣Spam)
$$
<br>
Essa suposição pode ser irrealista, mas o modelo funciona muito bem na prática e é usado em filtros de spam reais.

## Como calcular a probabilidade de uma mensagem ser spam?
O modelo usa o Teorema de Bayes:
<br>
<br>
$$
P(Spam∣Palavras) = \frac{P(Palavras∣Spam) × P(Spam)}{P(Palavras)}
$$
​<br>
Mas, como calcular P(Palavras∣Spam)?

Simples: multiplicamos a probabilidade de cada palavra individual aparecer em um spam.

Problema: Multiplicar muitas probabilidades pequenas pode causar problemas numéricos no computador, pois eles não processam bem os números de ponto flutuante muito próximos de 0.

Solução: Em vez de multiplicar, usamos logaritmos, pois somar logs evita números muito pequenos.
<br>
<br>
$$
logP(Spam∣Palavras)=∑logP(Palavra∣Spam)
$$

## O Problema das Probabilidades ZEROS e a Suavização
Se uma palavra nunca apareceu em mensagens de spam no nosso conjunto de treino, teríamos:
<br>
<br>
$$
P(bitcoin∣Spam)=0
$$
<br>
Isso faria o modelo atribuir probabilidade 0 a qualquer mensagem que contivesse essa palavra, o que não é desejável.

Para resolver isso, usamos a suavização de Laplace (ou pseudocontagem k):

$$
P(Palavra∣Spam)=\frac{ k + contagem da palavra em spams}{2k+total de spams}
$$
​
 
Isso garante que nenhuma palavra tenha probabilidade zero, permitindo ao modelo lidar melhor com palavras raras.

## Testando o Modelo



In [114]:
from nb import Message, NaiveBayesClassifier

messages = [Message("spam rules", is_spam=True),
            Message("ham rules", is_spam=False), # os hams são os não spams
            Message("hello ham", is_spam=False)]

model = NaiveBayesClassifier(k=0.5)
model.train(messages)

# Verificando se as contagens estão corretas
assert model.tokens == {"spam", "ham", "rules", "hello"}
assert model.spam_messages == 1
assert model.ham_messages == 2
assert model.token_spam_counts == {"spam": 1, "rules": 1}
assert model.token_ham_counts == {"ham": 2, "rules": 1, "hello": 1}

#### Precisamos analisar a lógica do Naive Bayes manualmente e verificar se obtemos o mesmo resultado. 

In [ ]:
import math

texto = "hello spam"

probs_se_spam = [
    (1 + 0.5) / (1 + 2 * 0.5),      # "spam"  (presente)
    1 - (0 + 0.5) / (1 + 2 * 0.5),  # "ham"   (ausente)
    1 - (1 + 0.5) / (1 + 2 * 0.5),  # "rules" (ausente)
    (0 + 0.5) / (1 + 2 * 0.5)       # "hello" (presente)
]

probs_se_ham = [
    (0 + 0.5) / (2 + 2 * 0.5),      # "spam"  (presente)
    1 - (2 + 0.5) / (2 + 2 * 0.5),  # "ham"   (ausente)
    1 - (1 + 0.5) / (2 + 2 * 0.5),  # "rules" (ausente)
    (1 + 0.5) / (2 + 2 * 0.5),      # "hello" (presente)
]

p_se_spam = math.exp(sum(math.log(p) for p in probs_se_spam))
p_se_ham = math.exp(sum(math.log(p) for p in probs_se_ham))

# Deve ser aproximadamente 0.83
print(model.predict(texto))
print(p_se_spam / (p_se_spam + p_se_ham))

0.8350515463917525
0.8350515463917525


### Parece que o modelo passou no teste. Agora, utilizaremos dados reais.
Os dados estão na pasta 'dados_spam', dentro dessa pasta tem outras três pastas: spam, easy_ham e hard_ham. Cada uma dessas pastas contém muitos e-mails, e cada e-mail fica em um arquivo. Para simplificar, usaremos apenas a linha de assunto de cada e-mail.

In [116]:
from typing import List
from glob import glob
from io import BytesIO
import requests
import tarfile
import warnings

warnings.filterwarnings('ignore')

URL = "https://spamassassin.apache.org/old/publiccorpus/"
ARQUIVOS = ["20021010_easy_ham.tar.bz2", "20021010_hard_ham.tar.bz2", "20021010_spam.tar.bz2"]
DIR_SAIDA = 'Naive Bayes/dados_spam'

for arquivo in ARQUIVOS:
    conteudo = requests.get(f"{URL}/{arquivo}").content
    fin = BytesIO(conteudo)
    with tarfile.open(fileobj=fin, mode='r:bz2') as tf:
        tf.extractall(DIR_SAIDA)

caminho = 'dados_spam/*/*'

dados: List[Message] = []

for filename in glob(caminho):
    is_spam = "ham" not in filename

    with open(filename, errors='ignore') as email_file:
        for line in email_file:
            if line.startswith("Subject:"):
                subject = line.lstrip("Subject: ")
                dados.append(Message(subject, is_spam))

                break

dados[:5]

[Message(text='Re: New Sequences Window\n', is_spam=False),
 Message(text='[zzzzteana] RE: Alexander\n', is_spam=False),
 Message(text='[zzzzteana] Moscow bomber\n', is_spam=False),
 Message(text="[IRR] Klez: The Virus That  Won't Die\n", is_spam=False),
 Message(text='Re: Insert signature\n', is_spam=False)]

In [117]:
from typing import TypeVar, Tuple
import random

X = TypeVar('X')  # tipo genérico para representar um ponto de dados

def dividir_dados(dados: List[X], prop: float) -> Tuple[List[X], List[X]]:
    """Divide os dados em frações [prop, 1 - prop]"""
    dados = dados[:]                    # Faz uma cópia rasa
    random.shuffle(dados)               # porque shuffle modifica a lista.
    corte = int(len(dados) * prop)  # Usa a prop para encontrar o ponto de corte
    return dados[:corte], dados[corte:]

random.seed(0) # para reproduzir os mesmos resultados
train_messages, test_messages = dividir_dados(dados, 0.75)

assert len(train_messages) == 0.75 * len(dados)
assert len(test_messages) == 0.25 * len(dados)
len(train_messages), len(test_messages)

(2475, 825)

In [118]:
from collections import Counter

modelo = NaiveBayesClassifier()
modelo.train(train_messages)

previsoes = [(mensagem, modelo.predict(mensagem.text)) for mensagem in test_messages]

confusion_matrix = Counter((mensagem.is_spam, probabilidade_spam > 0.5) for mensagem, probabilidade_spam in previsoes)
confusion_matrix

Counter({(False, False): 670,
         (True, True): 86,
         (True, False): 40,
         (False, True): 29})

Com nosso modelo naive bayes criado do zero, obtivemos 670 negativos verdadeiros (hams classificados como hams), 86 verdadeiros positivos (spams classificados como spams), 40 negativos falsos (spams classificados como hams) e 29 positivos falsos (hams classificados como spams). Então a precisão e a sensibilidade são:

In [119]:
precision = 86 / (86 +29)
recall = 86 / (86 + 40)
print(precision, recall)

0.7478260869565218 0.6825396825396826


E a acurácia é:

In [120]:
acertos = 0

for bool1, bool2 in confusion_matrix.keys(): 
    if bool1 == bool2: # verdadeiro positivo ou verdadeiro negativo, ou seja, os acertos
        acertos += confusion_matrix[(bool1, bool2)]

accuracy = acertos / len(test_messages)
accuracy

0.9163636363636364

### Agora, chegou a hora de comparar nosso modelo com o do scikit-learn.

Como o modelo BernoulliNB do scikit-learn não consegue trabalhar diretamente com textos, temos que usar o CountVectorizer com binary=True, que cria uma matriz onde cada palavra única vira uma coluna, e cada mensagem é representada por um vetor binário (1 se a palavra está presente, 0 se não está). Isso permite que o modelo use probabilidades para classificar mensagens como spam ou não.

In [121]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

X = [assunto.text for assunto in dados]  # Textos das mensagens
y = [assunto.is_spam for assunto in dados]  # Rótulos (spam ou não)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

# Convertendo o texto em uma matriz de presença/ausência de palavras
vectorizer = CountVectorizer(binary=True)  # binário pois é para BernoulliNB
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

nb_sklearn = BernoulliNB()
nb_sklearn.fit(X_train_vec, y_train)

previsoes_sklearn = nb_sklearn.predict(X_test_vec)

accuracy_sklearn = accuracy_score(y_test, previsoes_sklearn)
confusion_matrix_sklearn = confusion_matrix(y_test, previsoes_sklearn)
classification_report_sklearn = classification_report(y_test, previsoes_sklearn)

print(accuracy_sklearn, confusion_matrix_sklearn, classification_report_sklearn, sep="\n")

0.88
[[695   2]
 [ 97  31]]
              precision    recall  f1-score   support

       False       0.88      1.00      0.93       697
        True       0.94      0.24      0.39       128

    accuracy                           0.88       825
   macro avg       0.91      0.62      0.66       825
weighted avg       0.89      0.88      0.85       825



Nosso modelo Naive Bayes implementado do zero obteve uma acurácia de 91%, enquanto a versão do scikit-learn (BernoulliNB) alcançou 88%.

Isso mostra que nossa implementação conseguiu um desempenho ligeiramente melhor, o que pode ter sido influenciado por escolhas específicas no pré-processamento dos dados ou na suavização das probabilidades. No entanto, a diferença não é tão grande, e o modelo do scikit-learn tem a vantagem de ser otimizado, mais rápido e pronto para uso em aplicações reais.

Analisando a matriz de confusão do sklearn, vemos que o modelo classificou corretamente 695 hams (Verdadeiros negativos) e 31 spams (Verdadeiros positivos). No entanto, ele cometeu 97 erros ao classificar spams como hams (Falsos negativos), o que sugere que ele tem dificuldade em identificar spams corretamente.

O classification report reforça essa observação:

A precisão (precision) para a classe spam (True) é 0.94, o que significa que, das mensagens classificadas como spam, 94% eram realmente spams.
O recall para a classe spam é 0.24, indicando que o modelo só identificou corretamente 24% dos spams. Isso mostra que o modelo é conservador ao classificar uma mensagem como spam e acaba errando muito ao não identificar spams corretamente.
Para a classe ham (False), o recall é 1.00, ou seja, o modelo praticamente não classifica hams como spams, sendo muito confiável na detecção de mensagens legítimas.

A métrica F1-score (média harmônica entre precisão e recall) para spam ficou em 0.39, mostrando que o modelo tem dificuldades em equilibrar precisão e recall.

Em resumo, conseguimos validar que a abordagem teórica do Naive Bayes funciona bem na prática, e nossa implementação manual demonstrou um excelente desempenho, comparável a uma biblioteca amplamente usada.

Chegamos ao fim de mais uma implementação manual de um modelo!